Apply the classification algorithms Naive Bayes, Logistic regression and K-Nearest neighbours on the attached imdb dataset of review texts and review sentiment.

Convert the review text to Bag-of-Word (BOW) model with TF-IDF weights (text preprocessing should be applied first) and predict the review sentiment (positive or negative). Use label encoding to convert the sentiment feature to numerical values. The training/test split for the dataset should be 80/20.

print the accuracy score for each algorithm on the test dataset to find the most accurate model among the three created models.

In [ ]:
#importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import operator
import pandas as pd
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression

In [ ]:
#reading the datafile
data = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')


In [ ]:
#take a 10% sample of the dataset as recommended

imdb = data.sample(frac =.1)
imdb

In [ ]:

# replace values in sentiment feature from (positive, negative) to (1,0)
imdb['sentiment'].replace({'positive':1,'negative':0},inplace=True)


In [ ]:
#remove stopping words

stop = stopwords.words('english')
reviews_wo_stop = []

for rev in imdb['review']:
    text = []
    for word in rev.split():
        if word.lower() not in stop:
            text.append(word)
            
    reviews_wo_stop.append(text)

In [ ]:
#Convert the review text to Bag-of-Word (BOW) model with TF-IDF weights 
#(text preprocessing should be applied first) 

#text pre-processing and creating TFIDF vector.
#conver text to lowercase
#apply stemming
#apply lematization
#create the bag of words and tfidf vector

wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
def tokenize(str_input):
    
    words = re.sub(r"(?u)[^A-Za-z]", " ", str_input).lower().split(" ")
    words = [stemmer.stem(word) for word in words if len(word)>2]
    words = [wordnet_lemmatizer.lemmatize(word) for word in words if len(word)>2]
    return words


vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

vectors = vectorizer.fit_transform(imdb['review'])

feature_names = vectorizer.get_feature_names()

imdb_reviews_tfidf = pd.DataFrame(vectors.toarray(),columns=feature_names)

#print(imdb_reviews_tfidf.head())
#print(feature_names)
#print("number of words = ", len(feature_names))

In [ ]:

#and predict the review sentiment (positive or negative). 
#Use label encoding to convert the sentiment feature to numerical values. 
#The training/test split for the dataset should be 80/20.

#training/ test split
X=pd.DataFrame(vectors.toarray(),columns=feature_names) #imdb_reviews_tfidf
y = imdb.iloc[:,-1].values
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=0)



In [ ]:
#Applying Multinomial Naieve Bayes Classification 
#multinomial classification model  applied for text 
#Gaussian classification model applied for in case of using numbers. 


NB = MultinomialNB()
NB.fit(X_train,y_train)
y_NB=NB.predict(X_test)
print("Multinomial Accuracy=",accuracy_score(y_test,y_NB))
A=accuracy_score(y_test,y_NB) #SAVE MULTINOMIAL ACCURACY SCORE HERE FOR COMMPARISON 

In [ ]:
#P.S: This block of code takes nearly 3 hours to run
#Applying Nearest Neighbour Classification  
#numNeighbors = [1, 5, 10, 15, 20, 25, 30]
#trainAcc = []
#testAcc = []

#for k in numNeighbors:
#    clf = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)
#    clf.fit(X_train, y_train)
#    Y_predTrain = clf.predict(X_train)
#    Y_predTest = clf.predict(X_test)
#    trainAcc.append(accuracy_score(y_train, Y_predTrain))
#    testAcc.append(accuracy_score(y_test, Y_predTest))

#clf = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
#clf.fit(X_train, y_train)

#print("accuracy= ", clf.score(X_test,y_test))
#B= clf.score(X_test,y_test) #SAVE NEAREST NEIGHBOUR SCORE HERE FOR COMPARISON
#plt.plot(numNeighbors, trainAcc, 'ro-', numNeighbors, testAcc,'bv--')
#plt.legend(['Training Accuracy','Test Accuracy'])
#plt.xlabel('Number of neighbors')
#plt.ylabel('Accuracy')


In [ ]:
# Applying Logistic regression
C = [0.01, 0.1, 0.2, 0.5, 0.8, 1, 5, 10, 20, 50]
LRtrainAcc = []
LRtestAcc = []


for param in C:
    clf = linear_model.LogisticRegression(C=param)
    clf.fit(X_train, y_train)
    Y_predTrain = clf.predict(X_train)
    Y_predTest = clf.predict(X_test)
    LRtrainAcc.append(accuracy_score(y_train, Y_predTrain))
    LRtestAcc.append(accuracy_score(y_test, Y_predTest))

clf = linear_model.LogisticRegression(C=1.0)  
print("Accuracy= ",clf.fit(X_train, y_train).score(X_test,y_test))
C= clf.fit(X_train, y_train).score(X_test,y_test)


In [ ]:
#Comparison between all classification methods. 
#print the accuracy score for each algorithm 
#on the test dataset to find the most accurate model 
#among the three created models.



#due to very long run time of k-nearest neighbour I was not able to figure out the accuracy value. 
#Also, due to this code generating different result each time it runs 
#(because it is selecting a random sample from the reviews database )
#the Accuracy result is different after each run 

#to compare the three models, the best model is the one that provides the highest accuracy score

print('Final Results - Accuracy Scores:')
print ('Multinomial Accuracy= ', A)
#print ('K- Nearest Neighbor accuracy= ', B) # This line has been commented out because it takes a very long time to run.(3 hrs)
print ('Logistic Regresssion Accuracy= ',C)
print ("the highest value represents the most accurate model.")
